# BGC response of a cyclone vs. anticyclone 

**Region:** 46-52N and 15-23W

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import pop_tools
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cftime
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy
import dask
import os

Cannot write to data cache '/glade/p/cesmdata/cseg'. Will not be able to download remote data files. Use environment variable 'CESMDATAROOT' to specify another directory.


In [2]:
#from distributed import Client

In [3]:
#client = Client(n_workers=4, threads_per_worker=1)
#client

In [4]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client

cluster = NCARCluster()
cluster.scale(36) #adapt(minimum_jobs=1, maximum_jobs=40)
client = Client(cluster) # Connect this local process to remote workers
client

/glade/work/mclong/miniconda3/envs/exports/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://128.117.181.208:46794 Dashboard: https://jupyterhub.ucar.edu/dav/user/mclong/proxy/39941/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
grid = pop_tools.get_grid("POP_tx0.1v2")

In [6]:
domain = {"lat": [45, 52.5], "lon": [360 - 24, 360 - 14]}

J, _ = np.where((domain["lat"][0] <= grid.TLAT) & (grid.TLAT <= domain["lat"][1]))
_, I = np.where((domain["lon"][0] <= grid.TLONG) & (grid.TLONG <= domain["lon"][1]))

inx = slice(I.min(), I.max())
jnx = slice(J.min(), J.max())

In [7]:
zarr_stores = [
    "./data/g.e11.G.T62_t12.eco.006.pop.h.chl_surf.zarr/",
    "./data/g.e11.G.T62_t12.eco.006.pop.h.NCP.zarr/",
    "./data/g.e11.G.T62_t12.eco.006.pop.h.POC_flux_100m.zarr/",
    "./data/g.e11.G.T62_t12.eco.006.pop.h.SSH.zarr/",
]

In [8]:
dsets = [xr.open_zarr(store) for store in zarr_stores]
ds = xr.merge(dsets)

In [9]:
import scipy
import scipy.ndimage

ssh_mean = ds["SSH"].mean(dim="time")
ssh_mean_smth = xr.full_like(ssh_mean, fill_value=np.nan)
ssh_mean_smth.data = scipy.ndimage.filters.gaussian_filter(
    ssh_mean.values, [10.0, 10.0], mode="mirror"
)
ds["aSSH"] = ds["SSH"] - ssh_mean_smth

In [10]:
def date(ymd):
    """Return cftime object corresponding to model time at the end of the averaging interval"""
    num = cftime.date2num(
        cftime.DatetimeNoLeap(ymd[0], ymd[1], ymd[2]),
        units="days since 0001-01-01 00:00:00",
        calendar="noleap",
    )
    return cftime.num2date(
        num + 1, units="days since 0001-01-01 00:00:00", calendar="noleap"
    )


tracks = pd.read_csv("./data/gaube_structured_tracks_reformat.gzip", compression="gzip")
tracks["time"] = tracks[["year", "mon", "day"]].apply(date, axis=1)

In [11]:
contour_spec = dict(
    chl_surf=dict(levels=np.arange(0, 1.1, 0.1), extend="max"),
    NCP=dict(levels=np.arange(0, 50.5, 5), extend="both"),
    POC_flux_100m=dict(levels=np.arange(0, 10.5, 0.5), extend="max"),
)


def long_name_units(variable):
    if "long_name" in ds[variable].attrs:
        long_name = ds[variable].attrs["long_name"]
    else:
        long_name = variable

    if "units" in ds[variable].attrs:
        units = ds[variable].attrs["units"]
    else:
        units = None

    return long_name, units


grid = client.scatter(grid)
tracks = client.scatter(tracks)


def one_plot(grid, tracks, variable, time_value, figname=None):
    time_index = np.argwhere(ds.time.values == time_value).flatten()[0]
    da_var = ds[variable].isel(time=time_index)
    da_aSSH = ds["aSSH"].isel(time=time_index)
    time_level = ds.time.values[time_index]
    long_name, units = long_name_units(variable)

    fig = plt.figure(figsize=(16, 8))

    # 46-52N and 15-23W
    extent = [-24, -15, 45, 53]

    #    prj = ccrs.Mollweide(central_longitude=np.mean(extent[0:2]))
    prj = ccrs.Mercator(central_longitude=np.mean(extent[0:2]))
    ax = plt.axes(projection=prj)
    ax.set_extent(extent)

    lon = grid.TLONG.isel(nlat=jnx, nlon=inx)
    lat = grid.TLAT.isel(nlat=jnx, nlon=inx)

    cf = ax.contourf(
        lon, lat, da_var, transform=ccrs.PlateCarree(), **contour_spec[variable]
    )

    ax.contour(
        lon,
        lat,
        da_aSSH,
        colors="gray",
        levels=np.arange(-20, 24, 4),
        transform=ccrs.PlateCarree(),
    )

    cyclone = tracks.loc[(time_level == tracks.time) & (tracks.cyc == -1)]
    ax.plot(cyclone.x, cyclone.y, "rx", markersize=6, transform=ccrs.PlateCarree())
    ax.plot(
        cyclone.x,
        cyclone.y,
        "ro",
        markersize=8,
        markerfacecolor="none",
        transform=ccrs.PlateCarree(),
    )

    anticyc = tracks.loc[(time_level == tracks.time) & (tracks.cyc == 1)]
    ax.plot(anticyc.x, anticyc.y, "b.", markersize=4, transform=ccrs.PlateCarree())
    ax.plot(
        anticyc.x,
        anticyc.y,
        "bo",
        markersize=8,
        markerfacecolor="none",
        transform=ccrs.PlateCarree(),
    )

    cb = plt.colorbar(cf)

    ax.set_title(
        f"{time_level.year:04d}-{time_level.month:02d}-{time_level.day:02d}", loc="left"
    )
    ax.set_title(f"{long_name}", loc="right")

    if units is not None:
        cb.ax.set_title(f"{units}")

    ax.set_yticks(np.arange(45, 55, 2.5), crs=ccrs.PlateCarree())
    ax.set_xticks(np.arange(-24, -14, 2), crs=ccrs.PlateCarree())

    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

    if figname is not None:
        fig.savefig(figname, dpi=300, bbox_inches="tight")
        plt.close()

In [12]:
one_plot_delayed = dask.delayed(one_plot)

In [15]:
results = []
variables = [
    "NCP",
    "POC_flux_100m",
    "chl_surf",
]
for v in variables:
    os.makedirs(f"figs/movie_imgs/{v}", exist_ok=True)
    for i in range(ds.time.values.size):
        figo = one_plot_delayed(
            grid, tracks, v, ds.time.values[i], figname=f"figs/movie_imgs/{v}/{i:03d}.png"
        )
        results.append(figo)

In [16]:
%%time
results = dask.compute(*results)

CPU times: user 58.5 s, sys: 1.92 s, total: 1min
Wall time: 1min 59s
